In [1]:
import pandas as pd
path = "C:/Users/vince/user-behavior-predictor/data/raw/online_retail_II.xlsx"

sheets = pd.read_excel(path, sheet_name = None, engine = "openpyxl")
df = pd.concat (sheets.values(), ignore_index = True)

print(df.shape)
df.head()

(1067371, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [2]:
# Filter out returns and missing customer IDs
df_clean = df[df['Quantity'] > 0].dropna(subset=["Customer ID"])

#Rename and Cast
df_clean = ( df_clean.rename(columns={'Customer ID': 'CustomerID',}))

df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)

#Parse InvoiceDate
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

print('Cleaned data shape:', df_clean.shape)
df_clean.head()
    
            

Cleaned data shape: (805620, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom


In [3]:
#Aggregate per customer
user_agg = (df_clean.groupby('CustomerID')
            .agg( 
                total_orders = ('Invoice', 'nunique'),
                total_quantity = ('Quantity', 'sum'),
                first_date = ('InvoiceDate', 'min'),
                last_date = ('InvoiceDate', 'max'),
            )
            .reset_index()
            )
print ("User-level agg shape:", user_agg.shape)
user_agg.head()


User-level agg shape: (5881, 5)


,CustomerID,total_orders,total_quantity,first_date,last_date
0,12346,12,74285,2009-12-14 08:34:00,2011-01-18 10:01:00
1,12347,8,3286,2010-10-31 14:20:00,2011-12-07 15:52:00
2,12348,5,2714,2010-09-27 14:59:00,2011-09-25 13:13:00
3,12349,4,1624,2010-04-29 13:20:00,2011-11-21 09:51:00
4,12350,1,197,2011-02-02 16:01:00,2011-02-02 16:01:00


In [5]:
#Merge first_date back onto df_clean
df_with_first = df_clean.merge(user_agg[['CustomerID', 'first_date']], on='CustomerID')

#Compute dates since the first purchase
df_with_first['days_since_first'] = (df_with_first['InvoiceDate'] - df_with_first['first_date']).dt.days

#For each user, find the minimum days_since_first > 0 (First purchase)
repurchase = (df_with_first[df_with_first['days_since_first'] > 0]
              .groupby('CustomerID')['days_since_first']
              .min()
              .rename('days_to_repurchase')
              .reset_index())

#Assemble final user table with labels
user_final = (user_agg.merge(repurchase, on='CustomerID', how='left')
              .assign(did_repurchase_7d = lambda d: d['days_to_repurchase' ] <= 7)
              .fillna({'did_repurchase_7d': False}))
print('Final user table table:', user_final.shape)
user_final.head()
 



Final user table table: (5881, 7)


,CustomerID,total_orders,total_quantity,first_date,last_date,days_to_repurchase,did_repurchase_7d
0,12346,12,74285,2009-12-14 08:34:00,2011-01-18 10:01:00,4.0,True
1,12347,8,3286,2010-10-31 14:20:00,2011-12-07 15:52:00,37.0,False
2,12348,5,2714,2010-09-27 14:59:00,2011-09-25 13:13:00,80.0,False
3,12349,4,1624,2010-04-29 13:20:00,2011-11-21 09:51:00,18.0,False
4,12350,1,197,2011-02-02 16:01:00,2011-02-02 16:01:00,NaN,False
